# DeBussy Chordified (ver. 0.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

Credit for GPT2-RGA code used in this colab goes out @ Sashmark97 https://github.com/Sashmark97/midigen and @ Damon Gwinn https://github.com/gwinndr/MusicTransformer-Pytorch

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title nvidia-smi gpu check
!nvidia-smi

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/DeBussy

!pip install torch

!pip install tqdm
!pip install matplotlib
!pip install torch-summary

!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install pretty_midi

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import random
import copy

from collections import OrderedDict

from tqdm import tqdm

import matplotlib.pyplot as plt

from torchsummary import summary

print('Loading TMIDIX module...')
os.chdir('/content/DeBussy')

import TMIDIX
from GPT2RGAX import *

from midi2audio import FluidSynth
import pretty_midi
import librosa.display
from IPython.display import Audio

os.chdir('/content/')

# (MODEL LOAD)

In [ ]:
#@title Unzip pre-trained DeBussy Model
%cd /content/DeBussy/Model

print('=' * 70)
print('Unzipping pre-trained dataset-model...Please wait...')
print('=' * 70)

!cat /content/DeBussy/Model/DeBussy-Trained-Model.zip* > DeBussy-Trained-Model.zip
print('=' * 70)

!unzip -j DeBussy-Trained-Model.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/

In [ ]:
#@title Load/Reload the model

full_path_to_model_checkpoint = "/content/DeBussy/Model/DeBussy-Trained-Model.pth" #@param {type:"string"}

print('Loading the model...')
config = GPTConfig(640, 
                   512,
                   dim_feedforward=512,
                   n_layer=16, 
                   n_head=16, 
                   n_embd=512,
                   enable_rpr=True,
                   er_len=512)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT(config)

state_dict = torch.load(full_path_to_model_checkpoint, map_location=device)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.to(device)

model.eval()

print('Done!')

summary(model)

# (GENERATE)

# Load Custom MIDI / MIDI seed

In [ ]:
#@title Custom MIDI option
full_path_to_custom_MIDI = "/content/DeBussy/DeBussy-Seed-1.mid" #@param {type:"string"}

print('Loading custom MIDI file...')

score = TMIDIX.midi2ms_score(open(full_path_to_custom_MIDI, 'rb').read())

events_matrix1 = []

itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
        if event[0] == 'note' and event[3] != 9:
            events_matrix1.append(event)
    itrack += 1

# final processing...

events_matrix1.sort(key=lambda x: x[4], reverse=True) # Sort by pitch H -> L
events_matrix1.sort(key=lambda x: x[1]) # Then sort by start-times

if len(events_matrix1) > 0:

    # recalculating timings

    for e in events_matrix1:
        e[1] = int(e[1] / 10) # Time-shift
        e[2] = int(e[2] / 10) # Duration

    melody_chords = []

    pe = events_matrix1[0]

    cho = []

    for e in events_matrix1:

        if  e[1]-pe[1] == 0:
          cho.append([e[1]-pe[1], e[2], e[4], e[5]])
        else:
          melody_chords.append(cho)
          
          cho = []
        
          cho.append([e[1]-pe[1], e[2], e[4], e[5]])


        pe = e

melody_chords_f = []

for m in melody_chords:
  if len(m) == 1:
    noc = 638
  else:
    noc = 639

  chord = [noc]

  chord.extend([max(1, min(253, m[0][0]))])
  chord.extend([max(1, min(253, m[0][1]))+256])

  for mm in m:
    if mm[2]+512 not in chord:
      chord.append(mm[2]+512)

  melody_chords_f.append(chord)


inputs = []

for m in melody_chords_f:
  inputs.extend(m)


print('Done!')        

# Continuations

In [ ]:
#@title Single Continuation Block Generator

#@markdown NOTE: Play with the settings to get different results

number_of_prime_tokens = 128 #@param {type:"slider", min:32, max:384, step:8}
temperature = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}
show_stats = True #@param {type:"boolean"}

#===================================================================
print('=' * 70)
print('DeBussy Music Model Continuation Generator')
print('=' * 70)

print('Generation settings:')
print('=' * 70)
print('Number of prime tokens:', number_of_prime_tokens)
print('Model temperature:', temperature)

print('=' * 70)
print('Prepping...')


rand_seq = model.generate(torch.Tensor(inputs[:number_of_prime_tokens]), 
                                          target_seq_length=512,
                                          temperature=temperature,
                                          stop_token=640,
                                          verbose=show_stats)
  
out1 = rand_seq[0].cpu().numpy().tolist()


if len(out1) != 0:
    
    song = out1
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0
    son = [song[0]]
    for s in song[1:]:

        if s < 638:
          son.append(s)

        else:
          if len(son) > 2:

            time += son[1] * 10

            dur = (son[2] - 256) * 10
            
            channel = 0 # Piano

            if son[0] == 638:

                vel = 80
            else:
                vel = 110

            for ss in son[3:]:
              pitch = ss - 512
                              
              song_f.append(['note', time, dur, channel, pitch, vel ])
            
            son = []
            son.append(s)

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'DeBussy',  
                                                        output_file_name = '/content/DeBussy-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')


print('Displaying resulting composition...')
fname = '/content/DeBussy-Music-Composition'

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', fmin=1, hop_length=160, sr=16000, cmap=plt.cm.hot)
plt.title(fname)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# Congrats! You did it! :)